In [1]:
# Input information
# NOTE: the file name is just a file the author used for testing, the file is not included in the repository

# SA2-139E Ph1
meeting = {
    'file':  "2020.06.15 073045 email summary.csv",
    'title': "SA2-139E Ph1+Ph2",
    'dates': ['2020-05-31 12:00','2020-06-12 19:00']
}

# SA2-138E
#meeting = {
#    'file':  "2020.04.24 153334 email summary.csv",
#    'title': "SA2-138E",
#    'dates': ['2020-04-19 12:00','2020-04-24 19:00']
#}

# SA2-137E
#meeting = {
#    'file':  "2020.03.02 080314 email summary.csv",
#    'title': "SA2-137E",
#    'dates': ['2020-02-23 12:00','2020-02-28 19:00']
#}

In [2]:
# You will need pandas/numpy to run the first part of the script. Then plotly for the second part. You can install plotly via pip (pip install plotly)
# If the PNG export is not working, most probably orca is not installed. You can install it by executing in the Anaconda cmd: conda install -c plotly plotly-orca
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
# Create variables
email_dump    = meeting["file"]
title         = meeting["title"]
meeting_dates = meeting["dates"]

# Create DataFrame for analysis
df = pd.read_csv(
    email_dump,
    index_col=0,
    dtype=object)
df['date']    = pd.to_datetime(df['date'])
df['day']     = df['date'].dt.floor('d')
df['hour']    = df['date'].dt.floor('h')

def parse_email_company(x):
    try:
        email_address_split = x.split('@')
        company_domain      = email_address_split[-1]
        company_name_split  = company_domain.split('.')
        
        if 'COM.CN' in x:
            company_name = company_name_split[0]
        else:
            company_name = company_name_split[-2]
        
        if len(company_name) <= 4:
            company_name = company_name.upper()
        else:
            company_name = company_name.capitalize()
            
        if company_name=='COM':
            print(x)
        return company_name
    except:
        print('Error parsing email. Sender address: {0}'.format(x))
        return ''
df['company'] = df['sender address'].apply(parse_email_company)
# display(df.company.unique())

Error parsing email. Sender address: unknown@unknown
Error parsing email. Sender address: nan
Error parsing email. Sender address: nan
Error parsing email. Sender address: nan


array(['Ericsson', 'ETSI', 'Samsung', 'Nokia', 'CATT', 'VIVO', 'Unisoc',
       'LGE', 'Magenta', 'ZTE', 'Intel', 'Huawei', 'Qualcomm',
       'Nokia-sbell', 'Tencent', 'Vodafone', 'Nttdocomo', 'Cisco', 'ATT',
       'Interdigital', 'Matrixx', 'Chinamobile', 'Convidawireless',
       'OPPO', 'Kyocera', 'Broadcom', 'TNO', 'Ztetx', 'Charter', 'Orange',
       'Telecomitalia', '189', 'Mediatek', 'Alibaba-inc', 'SONY',
       'Futurewei', 'Lenovo', 'T-mobile', 'Chinatelecom', 'Philips',
       'Apple', 'Harting', 'Siemens', 'NEC', '163', 'Docomolab-euro',
       '126', 'Juniper', 'Gmail', 'CO', '', 'Allot', 'Sandvine',
       'Spirent', 'RE', 'Chinaunicom', 'Verizon', 'Foxmail',
       'Datangmobile', 'Motorola', 'Google', 'Cablelabs', 'Comcast',
       'Trideaworks', 'Firstnet', 'Fraunhofer', 'Thalesgroup'],
      dtype=object)

In [5]:
df

,date,ai,folder,subject,sender name,sender address,day,hour,company
0,2020-05-27 16:29:25,4.1,4.1,"[SA2#139E, AI#4.1, S2-2003599] Updating the UE...",Peter Hedman,peter.hedman@ERICSSON.COM,2020-05-27,2020-05-27 16:00:00,Ericsson
1,2020-05-27 16:59:21,4.1,4.1,"Re: [SA2#139E, AI#4.1, S2-2003599] Updating th...",Maurice Pope,Maurice.Pope@ETSI.ORG,2020-05-27,2020-05-27 16:00:00,ETSI
2,2020-05-28 14:49:34,4.1,4.1,"[SA2#139E, AI#4.1, S2-2004258] Handling manipu...",Lalith Kumar,lalith.kumar@SAMSUNG.COM,2020-05-28,2020-05-28 14:00:00,Samsung
3,2020-05-29 11:08:07,4.1,4.1,Re: Input required on topic for discussion on ...,"Thiebaut, Laurent (Nokia - FR/Paris-Saclay)",laurent.thiebaut@NOKIA.COM,2020-05-29,2020-05-29 11:00:00,Nokia
4,2020-05-29 11:46:27,4.1,4.1,答复: Input required on topic for discussion on ...,Hucheng Wang,wanghucheng@CATT.CN,2020-05-29,2020-05-29 11:00:00,CATT
...,...,...,...,...,...,...,...,...,...
6737,2020-06-11 16:43:38,4.2,4.2,"Re: [SA2#139E, AI#4.2, S2-2004635] LS from CT ...",Patrice Hédé,patrice.hede@HUAWEI.COM,2020-06-11,2020-06-11 16:00:00,Huawei
6738,2020-06-11 15:56:55,4.2,4.2,"Re: [SA2#139E, AI#4.2, S2-2004635] LS from CT ...",Peter Hedman,peter.hedman@ERICSSON.COM,2020-06-11,2020-06-11 15:00:00,Ericsson
6739,2020-06-11 00:23:36,4.2,4.2,"Re: [SA2#139E, AI#4.2, S2-2004336] LS from IEE...","Chandramouli, Devaki (Nokia - US/Dallas)",devaki.chandramouli@NOKIA.COM,2020-06-11,2020-06-11 00:00:00,Nokia
6740,2020-06-10 16:41:17,4.2,4.2,"Re: [SA2#139E, AI#4.2, S2-2004635] LS from CT ...",Patrice Hédé,patrice.hede@HUAWEI.COM,2020-06-10,2020-06-10 16:00:00,Huawei


In [6]:
email_count_per_ai   = df.groupby('ai').count()
email_count_per_day  = df.groupby('day').count()
email_count_per_hour = df.groupby('hour').count()

In [7]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count = df.groupby('sender name').count().sort_values(by='date', ascending=False)
# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count['ranking'] = np.arange(len(emails_count)) + 1
emails_count['emails'] = emails_count['date']
display(emails_count[['ranking', 'emails']].iloc[0:25,:])
pd.options.display.max_rows = 60

,ranking,emails
sender name,,
"Stojanovski, Saso",1,213
"Casati, Alessio (Nokia - GB)",2,187
LaeYoung Kim,3,173
"Shan, Chang Hong",4,161
"Thiebaut, Laurent (Nokia - FR/Paris-Saclay)",5,157
Haris Zisimopoulos,6,137
zhuqianghua,7,128
"Chandramouli, Devaki (Nokia - US/Dallas)",8,124
"Hietalahti, Hannu (Nokia - FI/Oulu)",9,121


In [93]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count_company = df.groupby('company').count().sort_values(by='date', ascending=False)

# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count_company['ranking'] = np.arange(len(emails_count_company)) + 1
emails_count_company['emails'] = emails_count_company['date']
display(emails_count_company[['ranking', 'emails']].iloc[0:20,:])
pd.options.display.max_rows = 60

,ranking,emails
company,,
Huawei,1,855
Nokia,2,805
Ericsson,3,765
Qualcomm,4,628
Samsung,5,535
Intel,6,430
LGE,7,266
ZTE,8,202
VIVO,9,175


In [95]:
# Ranking based on emails per delegate
active_delegate_list = pd.DataFrame()
grouped_df = df.groupby(['company', 'sender name']).count()
active_delegate_list['company']       = grouped_df.index.get_level_values(0).copy()
active_delegate_list['delegate name'] = grouped_df.index.get_level_values(1).copy()
# display(active_delegate_list)
active_delegate_count = active_delegate_list.groupby('company').count()
# display(active_delegate_count)
# display(emails_count_company)

emails_per_active_delegate_count = emails_count_company['emails']/active_delegate_count['delegate name']
emails_count_company['emails/active delegate'] = emails_per_active_delegate_count
emails_count_company['active delegates'] = active_delegate_count['delegate name']
emails_count_company['total emails'] = emails_count_company['emails']
emails_count_company_resort = emails_count_company.sort_values(by='emails/active delegate', ascending=False)
emails_count_company_resort['ranking'] = np.arange(len(emails_count_company)) + 1
display(emails_count_company_resort[['ranking', 'emails/active delegate', 'active delegates', 'total emails']].iloc[0:20,:])

,ranking,emails/active delegate,active delegates,total emails
company,,,,
Nokia,1,115.000000,7,805
Intel,2,107.500000,4,430
Qualcomm,3,89.714286,7,628
Nokia-sbell,4,85.000000,1,85
LGE,5,66.500000,4,266
Ericsson,6,63.750000,12,765
Magenta,7,56.333333,3,169
Ztetx,8,55.000000,1,55
Huawei,9,53.437500,16,855


In [80]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
data = go.Bar(
    x=email_count_per_ai.index, 
    y=email_count_per_ai['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='{1}: Emails/AI (total {0})'.format(email_count_per_ai['date'].sum(), title))
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
data = go.Bar(
    x=email_count_per_day.index, 
    y=email_count_per_day['date'])
layout = go.Layout(
    title='Emails per day (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m',
    xaxis_range = meeting_dates,
    xaxis = { 'dtick': 1000*3600*24 })
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per day")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Data source for emails/day per AI
email_count_per_day_and_ai = df.groupby(['ai', 'day']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_day_and_ai[email_count_per_day_and_ai.index.isin([ai], level=0)]['date']

layout = go.Layout(
    title='{1}: Emails/day (total {0})'.format(email_count_per_ai['date'].sum(), title),
    xaxis_tickformat = '%d.%m',
    xaxis_range = meeting_dates,
    xaxis = { 'dtick': 1000*3600*24 } )

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()

fig_title = "{0} {1}".format(title, "Emails per day and AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Deactivated cell (we already have the plot per AI)
data = go.Bar(
    x=email_count_per_hour.index, 
    y=email_count_per_hour['date'])
layout = go.Layout(
    title='Emails per hour (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m  %Hh',
    xaxis_range = meeting_dates)
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per hour")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Data source for emails/h per AI
email_count_per_hour_and_ai = df.groupby(['ai', 'hour']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_hour_and_ai[email_count_per_hour_and_ai.index.isin([ai], level=0)]['date']

layout = go.Layout(
    title='{1}: Emails/hour (total {0})'.format(email_count_per_ai['date'].sum(), title),
    xaxis_tickformat = '%d.%m %Hh',
    xaxis_range = meeting_dates)

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()

fig_title = "{0} {1}".format(title, "Emails per hour and AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")